# Datu tīrīšana


Kārtošu Siguldas novada pašvaldības ceļu un ielu sarakstu, kas ir sadalīts četrās Excel darblapās: A, B, C ceļi, un ielas. Katra no šīm 4 darblapām sastāv no vairākām tabulām ar virsrakstiem un kopsavilkumiem, kur ceļi un ielas ir salikti pēc teritoriālā sadalījuma. Daudzi ceļi un ielas ir vēl sīkāk sadalīti posmos, kur nosaukums ir aizpildīts tikai pirmajam posmam.

Apvienošu datus vienā tabulā, teritoriālo iedalījumu iznesīšu atsevišķā kolonnā, izdalīšu atzarus un katram ceļam/ielai pievienošu ID numurus (jo nosaukumi atkārtojas).

Dati xlsx formātā pieejami šeit: https://www.sigulda.lv/public/download.php?id=6266 

In [1]:
import pandas as pd

In [2]:
# ielādējot datus no darblapām norādu tikai nepieciešamās kolonnas piešķirot tām nosaukumus
# norādu neielādēt rindas, kur visas vērtības ir tukšas

url = 'https://www.sigulda.lv/public/download.php?id=6266'
kol_nosaukumi = ['nosaukums', 'uzturesanas_klase', 'virsraksts', 'garums', 'segums']

df1 = pd.read_excel(url, 
                    sheet_name='A ceļi', 
                    usecols='B:D, F, H', 
                    names=kol_nosaukumi)\
                    .dropna(how='all')

df2 = pd.read_excel(url, 
                    sheet_name='B ceļi', 
                    usecols='B:D, F, H', 
                    names=kol_nosaukumi)\
                    .dropna(how='all')

df3 = pd.read_excel(url, 
                    sheet_name='C ceļi', 
                    usecols='B:D, F, H', 
                    names=kol_nosaukumi)\
                    .dropna(how='all')

df4 = pd.read_excel(url, 
                    sheet_name='Ielas', 
                    usecols='B:D, F, H', 
                    names=kol_nosaukumi)\
                    .dropna(how='all')


# izveidoju jaunu kolonnu priekš ceļu un iela grupas
df1.insert(1, 'grupa', 'A ceļš')
df2.insert(1, 'grupa', 'B ceļš')
df3.insert(1, 'grupa', 'C ceļš')
df4.insert(1, 'grupa', 'Iela')


# apvienoju kopā
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [3]:
# pārnesīšu teritoriālo sadalījumu jaunajā kolonnā


# no kolonnas "virsraksts" pārnesu divus pēdējos vārdus (teritorijas nosaukums) uz jaunizveidoto kolonnu "teritorija"
# un izdzēšu vairs nevajadzīgu kolonnu
split = df['virsraksts'].str.rsplit(n=2)
df['teritorija'] = split.str[1] + " " + split.str[2]
df = df.drop('virsraksts', axis=1)


# apskatu unikālās vērtības jaunajā kolonnā
df.teritorija.unique()

array(['Siguldas pagastā', 'raksturojošie parametri', nan,
       'Allažu pagastā', 'Mālpils pagastā', 'Inčukalna pagastā',
       'Krimuldas pagastā', 'Lēdurgas pagastā', 'Rihards Keišs',
       'uzvārds )', 'domes izpilddirektors', 'Mores pagastā',
       'Siguldas pilsētā', 'pilsētā (Peltēs)', 'pilsētā (Ķiparos)',
       'pilsētā (Kalnabeitēs)', 'Matiņu ciemā', 'Vējupītes ciemā',
       'Jūdažu ciemā', 'Mores ciemā', 'Allažu ciemā',
       '(Siguldas pagastā)', 'Mālpils ciemā', 'Upmalu ciemā',
       'Sidgundas ciemā', 'Inčukalna ciemā', 'Gaujas ciemā',
       '(Inčukalna pagastā)', 'Krustiņu ciemā', 'Kļavu ciemā',
       'Raganas ciemā', 'Inciema ciemā', 'Sunīšu ciemā', 'Turaidas ciemā',
       'Lēdurgas ciemā', 'Lodes ciemā'], dtype=object)

In [4]:
# koriģēju locījumu: "pagastā" nomainu uz "pagasts", un "ciemā" nomainu uz "ciems"
df['teritorija'] = df['teritorija'].replace(r'(.*)pagastā$', r'\1pagasts', regex=True)
df['teritorija'] = df['teritorija'].replace(r'(.*)ciemā$', r'\1ciems', regex=True)


# koriģēju pilsētas un to rajonu nosaukumus
df['teritorija'] = df['teritorija'].replace({'Siguldas pilsētā': 'Siguldas pilsēta', 
                                             'pilsētā (Peltēs)': 'Siguldas pilsēta (Peltes)', 
                                             'pilsētā (Ķiparos)': 'Siguldas pilsēta (Ķipari)', 
                                             'pilsētā (Kalnabeitēs)': 'Siguldas pilsēta (Kalnabeites)'})


# koriģēju Egļupes ciema nosaukumu (atrodas divos pagastos)
df['teritorija'] = df['teritorija'].replace({'(Siguldas pagastā)': 'Egļupes ciems (Allažu pag.)', 
                                             '(Inčukalna pagastā)': 'Egļupes ciems (Inčukalna pag.)'})


In [5]:
# izdzēšu visus nesaistītos ierakstus šajā kolonnā (kur nav pieminēts pagasts, pilsēta, vai ciems)
df['teritorija'] = df['teritorija'].replace(r'^(?!.*pagasts|.*pilsēta|.*ciems).*$', pd.NA, regex=True)


# un aizpildu tukšas vērtības
df['teritorija'] = df['teritorija'].ffill()

In [6]:
# atskatu uzturēšanas klašu unikālās vērtības 
df.uzturesanas_klase.unique()

array([nan, 'Uzturēšanas klase', 'C', 'D', 'B'], dtype=object)

In [7]:
# izņemu rindas, kur nav norādīta uzturēšanas klase (tās būs visas liekas rindas)
uztur_klases = ['C', 'D', 'B']
df = df.loc[df['uzturesanas_klase'].isin(uztur_klases)]

In [8]:
# pārliecinos, ka nosaukumos nav liekas atstarpes (kuras aizpildu ar tukšu vērtību)
df['nosaukums'] = df['nosaukums'].str.strip().replace('', pd.NA)

# aizpildu secīgi nosaukumus (rindas vēl nemainīja savu sākotnējo secību)
df.loc[:, 'nosaukums'] = df['nosaukums'].ffill()

In [9]:
# apskatu unikālās vērtības kolonnā 'segums'
df.segums.unique()

array(['grants', 'melnais', 'cits segums', 'bruģis', 'dubultā virsma',
       'Grants'], dtype=object)

In [10]:
# koriģēju dublikātu
df['segums'] = df['segums'].replace('Grants', 'grants')

In [11]:
# izdalīšu atzarus atsevišķā kolonnā
# atzaru posmiem secība ir vienmēr pēdējā starp viena ceļa/ielas posmiem

df.loc[df['nosaukums'].str.contains('atzars', case=False), 'atzars'] = True
df.loc[~df['nosaukums'].str.contains('atzars', case=False), 'atzars'] = False



In [12]:
# pārliecinos, ka nav divi atzaru posmi viens pēc otra:
# pabīdu indeksu par vienu vienību (lai redzētu rindas tieši pirms katra atzara) un vai nosaukumā ir pieminēs vārds atzars

df.loc[(df['atzars'].shift(-1) == True) & (df['nosaukums'].str.contains('atzars', case=False))]

,nosaukums,grupa,uzturesanas_klase,garums,segums,teritorija,atzars


In [13]:
# nomainu atzaru nosaukumus uz attiecīgā ceļa/ielas nosaukumu:
# indeksu pabīdu pretējā virzienā lai pārņemtu nosaukumu no iepriekšējās rindas

df.loc[df['atzars'] == True, 'nosaukums'] = df['nosaukums'].shift(1)

In [14]:
# katram ceļam/ielai piešķiru unikālu ID numuru
id = df['nosaukums'] + df['teritorija']
df['id'] = id.factorize()[0] + 1

# ceļu un ielu kopējs skaits
df.id.unique().shape

(767,)

In [15]:
# apskatu datu tipus
df.dtypes

nosaukums            object
grupa                object
uzturesanas_klase    object
garums               object
segums               object
teritorija           object
atzars               object
id                    int64
dtype: object

In [16]:
# nomainu uz float formātu
df['garums'] = pd.to_numeric(df['garums'], errors="coerce")
df.dtypes

nosaukums             object
grupa                 object
uzturesanas_klase     object
garums               float64
segums                object
teritorija            object
atzars                object
id                     int64
dtype: object

In [17]:
# pārliecinos, ka nav palikušas tukšas vērtības
df.isnull().sum()

nosaukums            0
grupa                0
uzturesanas_klase    0
garums               0
segums               0
teritorija           0
atzars               0
id                   0
dtype: int64

In [18]:
# satīrīti dati

df = df.reset_index(drop=True)
df

,nosaukums,grupa,uzturesanas_klase,garums,segums,teritorija,atzars,id
0,9410 A2- Kundziņi- Muldas,A ceļš,C,3.130,grants,Siguldas pagasts,False,1
1,9410 A2- Kundziņi- Muldas,A ceļš,C,0.880,grants,Siguldas pagasts,False,1
2,9420 Stūrīši- Ezermalas (Jūdaži),A ceļš,D,1.420,melnais,Siguldas pagasts,False,2
3,9433 P8- Limbas (Limbu ceļš),A ceļš,C,0.970,melnais,Siguldas pagasts,False,3
4,4205 Egļupes ceļš,A ceļš,C,0.420,grants,Allažu pagasts,False,4
...,...,...,...,...,...,...,...,...
1187,Upes iela,Iela,D,0.195,grants,Lēdurgas ciems,False,764
1188,Upeskalna iela,Iela,D,0.177,grants,Lēdurgas ciems,False,765
1189,Upeskalna iela,Iela,D,0.053,grants,Lēdurgas ciems,False,765
1190,Vītolu iela,Iela,D,0.200,grants,Lodes ciems,False,766
